
## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bokeh.models import FixedTicker
from utils import *

from dotenv import load_dotenv
import geopandas as gpd
import geoviews as gv
import geoviews.feature as gf
import hvplot.pandas
import hvplot.xarray
import os
import pandas as pd
import seaborn as sns
import xagg as xa
import xarray as xr

## Set environment variables

In [ ]:
load_dotenv()

os.environ['OAP_DATA_DIR'] = "/home/daniele/Documents/CHD/CHD_Python_projects/pa-aa-toolbox-folder/"
os.environ['IRI_AUTH'] = os.getenv("IRI_AUTH")
tchad_config_file = "config/countries/tcd_adm0.yaml"

## Create country configuration

In [ ]:
country_config, geobb = create_area(tchad_config_file)

## Create IRI terciles

In [ ]:
ds_iri_terciles = iri_terciles_create(
    country_config=country_config, 
    geobb=geobb,
    only_dominant=False
)
ds_iri_tercile_dominant = calculate_dominant_tercile(ds_iri_terciles)

## Create CHIRPS terciles

In [ ]:
ds_chirps_terciles = chirps_terciles_create(
    country_config=country_config, 
    geobb=geobb,
    adapt_coordinates=True,
    ds_ref=ds_iri_terciles
)
ds_chirps_tercile_dominant = calculate_dominant_tercile(ds_chirps_terciles)

## Create ECMWF terciles

In [ ]:
ds_ecmwf_terciles = ecmwf_terciles_create(
    country_config=country_config, 
    geobb=geobb,
    adapt_coordinates=True,
    ds_ref=ds_iri_terciles,
)
ds_ecmwf_tercile_dominant = calculate_dominant_tercile(ds_ecmwf_terciles)

## Adapt coordinates for IRI

In [ ]:
ds_iri_tercile_dominant = ds_iri_tercile_dominant.sel(latitude=ds_ecmwf_tercile_dominant.latitude)
ds_iri_tercile_dominant = ds_iri_tercile_dominant.sel(longitude=ds_ecmwf_tercile_dominant.longitude)

## Visualise terciles

In [ ]:
plot_tercile(
    ds_chirps_tercile_dominant,
    "3",
    "CHIRPS"
)

In [ ]:
plot_tercile(
    ds_iri_tercile_dominant,
    "3",
    "IRI"
)

In [ ]:
plot_tercile(
    ds_ecmwf_tercile_dominant,
    "3",
    "ECMWF"
)

## Calculate metrics: accuracy and F1 score

### Full results

In [ ]:
score_dataset = compute_scores(
    ds_chirps_tercile_dominant=ds_chirps_tercile_dominant,
    ds_iri_tercile_dominant=ds_iri_tercile_dominant,
    ds_ecmwf_tercile_dominant=ds_ecmwf_tercile_dominant,
    allign_timeframes=False,
    verbose=True,
)

### Results for restricted time

In [ ]:
score_dataset_allign_timeframe = compute_scores(
    ds_chirps_tercile_dominant=ds_chirps_tercile_dominant,
    ds_iri_tercile_dominant=ds_iri_tercile_dominant,
    ds_ecmwf_tercile_dominant=ds_ecmwf_tercile_dominant,
    allign_timeframes=False,
    verbose=True,
)

## Plot raster scores

In [ ]:
plot_scores(
    score_dataset=score_dataset,
    season='JJA',
    ltime="3",
    dataset="IRI",
    metric='Accuracy'
)

## Sahel Strip Analysis

### Load polyfon

In [ ]:
tchad_gdf = gpd.read_file('./data/tcd_sahel_strip_ocha.shp')

### Aggregate scores

In [ ]:
df = aggregate_scores(
    score_dataset=score_dataset,
    gdf=tchad_gdf,
    method="pandas",
)

In [ ]:
df_alligned_timeframe = aggregate_scores(
    score_dataset=score_dataset_allign_timeframe,
    gdf=tchad_gdf,
    method="pandas",
)

### Plot aggregated scores (barplots)

In [ ]:
plot_aggregated_scores_barplot(
    df=df, 
    timeframe='full timeframes', 
    score_metric='f1 score', 
    threshold='Lower tercile, probability greater than 0.4', 
    leadtime="3"
)
        

In [ ]:
plot_aggregated_scores_barplot(
    df=df_alligned_timeframe, 
    timeframe='alligned timeframes', 
    score_metric='f1 score', 
    threshold='Lower tercile, probability greater than 0.4', 
    leadtime="3"
)

### Plot aggregated scores (heatmaps)

In [ ]:
for season in ['MJJ', 'JJA', 'JAS', 'ASO']:
    for dataset in ['ecmwf', 'iri']:
        plot_aggregated_scores_heatmap(
            df=df,
            dataset=dataset,
            season=season
        )